In [1]:
import docx
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from markdown_it import MarkdownIt
from mdit_py_plugins.dollarmath import dollarmath_plugin

# --- Вхідний текст у форматі Markdown ---
markdown_text = r"""
Запропоновано формальний, але інтуїтивно зрозумілий алгоритм, який автоматично формує дерево рівнів / класів, спираючись на (і) відстані між векторними представленнями класів, (іі) величину плутанини між ними та (ііі) прості порогові правила. Алгоритм узгоджує емпіричні спостереження авторів щодо трирівневої схеми «H / V → T / B / O → G / M / Z»  й узагальнює їх до автоматично масштабованої процедури.

---

# Запропонований метод

**Вхідні дані**

| Позначення                         | Розмірність                                                  | Пояснення                                                             |
| ---------------------------------- | ------------------------------------------------------------ | --------------------------------------------------------------------- |
| $D=\{(x_i,y_i)\}_{i=1}^{N}$        | $N\times$(зображення + мітка)                                | Навчальний набір кадрів БПЛА з їхніми початковими мітками             |
| $L=\{l_1,\dots ,l_m\}$             | $m$                                                          | **Базові класи** (тобто найдрібніші, що вже не діляться)              |
| $m$                                | скаляр                                                       | Кількість базових класів                                              |
| $B$                                | функція $\mathbb{R}^{H\times W\times3}\!\to\!\mathbb{R}^{d}$ | Back-bone, який перетворює ROI у вектор ознак розмірності $d$         |
| $λ\in[0,1]$                        | скаляр                                                       | Вага між «просторовою відстанню» й «частотою помилкових сплутувань»   |
| $δ_{\min}$                         | скаляр                                                       | Мінімальна відстань між кластерами, за якої їх ще розділяємо          |
| $ε_{\max}$                         | скаляр                                                       | Максимально допустима частка помилкових сплутувань усередині кластера |
| $Θ=(\text{FPS}_{\min},\,T_{\max})$ | вектор $2\times1$                                            | Мінімальна швидкість (кадрів/с) та максимальна латентність (мс)       |

---

## Крок 1: Витяг ознак для усіх ROI

1.1 Для кожного зображення $x_i$ виконуємо первинне детектування (будь‑який SOTA‑детектор) та отримуємо множину ROI $R_i=\{r_{i1},\dots ,r_{ik}\}$.

1.2 Для кожного ROI обчислюємо вектор ознак

$$
f_{ij}=B(r_{ij})\in\mathbb{R}^{d}.
$$

---

## Крок 2: Побудова «профілю» класу

2.1 Для кожного атомарного класу $l\in L$ збираємо множину ознак $F_l=\{f_{ij}\mid y_{ij}=l\}$.

2.2 Обчислюємо **центроїд**

$$
\mu_l=\frac{1}{|F_l|}\sum_{f\in F_l}f.
\tag{1}
$$

2.3 Одноразово навчаємо одноетапний класифікатор без ієрархії $C^{\text{flat}}$ (наприклад, простий MLP) на всіх $f_{ij}$ та формуємо **матрицю плутанини**

$$
\Omega\in[0,1]^{m\times m},\quad
\Omega_{pq}=\frac{\text{FP}(p\to q)}{\sum_{q'}\text{FP}(p\to q')},
\tag{2}
$$

де $\text{FP}(p\to q)$ — кількість неправильних присвоєнь $p\mapsto q$.

Тут під одноетапним класифікатором без ієрархії $C^{\text{flat}}$ розуміємо модель, що одразу прогнозує один з усіх $m$ базових класів без ієрархії.

---

## Крок 3: Визначення міри близькості між класами

Комбінуємо геометричну й емпіричну інформацію:

$$
M_{pq}=λ\cdot\|\mu_p-\mu_q\|_2+
(1-λ)\cdot(1-\Omega_{pq}-\Omega_{qp}),
\qquad p\neq q.
\tag{3}
$$

> *Чим більшим є $M_{pq}$ — тим легше класи розділити.*

---

## Крок 4: Ієрархічне агломеративне групування

4.1 Будуємо повнозважений граф з вагами $M_{pq}$ та застосовуємо **agglomerative clustering (average linkage)**, отримуючи дендрограму $T$.

4.2 Переходимо зверху донизу: для кожної внутрішньої вершини $S\subseteq L$ перевіряємо:

$$
\min_{p\neq q\in S}M_{pq} < δ_{\min }\;\lor\;
\frac{\sum_{p,q\in S,\,p\neq q}\Omega_{pq}}{|S|(|S|-1)} > ε_{\max }.
\tag{4}
$$

* Якщо **умова виконується** → **не розділяємо** (класи лишаються у цьому ж листі).
* Якщо **умова не виконується** → **ділимо** вершину на її дві дочірні кластери й повторюємо перевірку.

У результаті маємо $K$ рівнів $\{L_1,\dots ,L_K\}$ з дедалі дрібнішими класами; алгоритм природно повторює логіку **H/V → T/B/O → G/M/Z** з рукопису , але без ручного втручання.

---

## Крок 5. Призначення моделей на кожен рівень ієрархії

> **Мета:** для кожного рівня $k$ вибрати пару **детектор $D_k$** і **класифікатор $C_k$** так, щоб:
>
> 1. виконувалися ресурсні обмеження $Θ=(\text{FPS}_{\min},\,T_{\max})$;
> 2. модель залишалася придатною до малого/великого числа класів та розміру об’єктів.

### 5.1  Вхідні параметри (усі легко вимірювані)

| Змінна           | Розмірність / Тип  | Як отримати                                                                        | Що означає                                                                  |                                                                    |                                                       |
| ---------------- | ------------------ | ---------------------------------------------------------------------------------- | --------------------------------------------------------------------------- | ------------------------------------------------------------------ | ----------------------------------------------------- |
| (                | L\_k               | )                                                                                  | скаляр (ціле)                                                               | Злічити кількість унікальних підкласів, що залишилися на рівні $k$ | Скільки категорій треба розрізняти на поточному рівні |
| $\bar{s}_k$      | скаляр (пікселі)   | Взяти медіану значень $\sqrt{w_{ij}h_{ij}}$ для всіх ROI рівня $k$                 | Середній лінійний розмір об’єктів, які обробляє рівень                      |                                                                    |                                                       |
| $FPS_{\min}$     | скаляр (кадри / с) | Задано технічним завданням                                                         | Мінімально допустима швидкість системи в робочому режимі                    |                                                                    |                                                       |
| $T_{\max}$       | скаляр (мс)        | Задано технічним завданням                                                         | Гранична сумарна латентність повного каскаду                                |                                                                    |                                                       |
| $C_{\text{GPU}}$ | скаляр (TFLOPS)    | Паспортна потужність доступного GPU (або фактична, виміряна утилітою `nvidia-smi`) | Обчислювальний ресурс, на який спирається вибір «важкої» чи «легкої» моделі |                                                                    |                                                       |

### 5.2 Дерево рішень (детермінований алгоритм)

pseudocode
if |L_k| ≤ 2 and \bar{s}_k ≥ 48:
    D_k = YOLOv11-l
    C_k = MLP (1×256)
elif 3 ≤ |L_k| ≤ 6 and \bar{s}_k ≥ 32:
    D_k = YOLOv11-m
    C_k = FT-Transformer (d=512, h=8, 2 blocks)
elif |L_k| > 6 or \bar{s}_k < 32:
    D_k = RT-DETR-Tiny
    C_k = FT-Transformer (d=768, h=12, 4 blocks)
# ресурсна деградація
if C_GPU < 5:
    D_k = YOLO-Nano
    C_k = MobileNet-MLP

### 5.3 Правила корекції

1. **Перевірка латентності:** якщо сумарна латентність рівня $T_k > T_{\max}/K$, знижуємо варіант на один рівень у дереві.
2. **Малі об’єкти** $(\bar{s}_k < 16$ px): додаємо попередню SR-підвищувальну мережу (наприклад, ESRGAN ×2) перед $D_k$.
3. **FPS-тест:** якщо фактичний FPS $<\text{FPS}_{\min}$, замінюємо $C_k$ на компактний MLP і повторюємо тест.

### 5.4 Відтворюваність

* **Порогові значення** (48 px, 32 px, 16 px, TFLOPS = 5) обрано на базі емпіричних меж з розд. 2.5 рукопису; у реплікації їх можна варіювати, але сам алгоритм (порівняння « > / ≤ ») залишається сталим.
* **Вхідні змінні** $|L_k|$, $\bar{s}_k$, $C_{\text{GPU}}$ вимірюються однозначно; отже для тих самих даних і «заліза» вибір моделей повториться.
* Таблиця/псевдокод вище може бути реалізована в два рядки Python (`if-elif-else`), тому pipeline легко інтегрується у CI.

**Вихід:** для кожного рівня $k$ повертаємо пару $(D_k,\;C_k)$, гарантувавши, що вся ієрархія задовольняє $Θ$ і не потребує ручного тюнінгу.

---

## Крок 6: Послідовне (ззовні → всередину) навчання

6.1 Навчаємо $(D_1,F_1,C_1)$ на всьому датасеті.

6.2 Для кожного наступного рівня $k$: використовуємо тільки ті ROI, які попередній рівень відніс до надкласу, та навчаємо $(D_k,F_k,C_k)$.

6.3 Для побудови **конкатенованого вектора ознак** шукаємо підмножину шарів $\mathcal{S}$ максимізуючи

$$
\Phi(\mathcal{S})=\min_{p\neq q\in L_k}\|\mu_{p,\mathcal{S}}-\mu_{q,\mathcal{S}}\|_2,
\tag{7}
$$

аналогічно вибору у табл. 1 і рис. 5 рукопису .

---

## Крок 7: Калібрування порогів та перевірка ресурсних обмежень

7.1 Для кожного $C_k$ калібруємо поріг довіри $p_k^\star$ за критерієм Youden J.

7.2 Обчислюємо сумарну латентність і FPS; якщо не виконує $Θ$ — збільшуємо $δ_{\min }$ (отримаємо менше рівнів) та/або замінюємо моделі згідно (5)–(6).

---

## Крок 8: Експорт структури

Формуємо JSON / YAML‑файл

yaml:
levels:
  - id: 1
    classes: ["H","V"]
    detector: YOLOv11-l
    classifier: FT-Transformer
    threshold: 0.82
  - id: 2
    parent_class: "V"
    classes: ["T","B","O"]
    detector: YOLOv11-m
    classifier: FT-Transformer
    threshold: 0.80
  - id: 3
    parent_class: "O"
    classes: ["G","M","Z"]
    detector: YOLOv11-s
    classifier: MLP
    threshold: 0.78

* збережені ваги моделей.

---

**Вихідні дані:**

* Дерево ієрархії $H=(L_1,\dots ,L_K)$ з моделями $\{(D_k,F_k,C_k)\}_{k=1}^{K}$.
* Конфігураційний файл для розгортання на БПЛА чи наземній станції.
* Аналітичний звіт (таблиці метрик, граф FPS‑vs‑F1, дендрограма).

---

## Опис усіх позначень

| Позначення | Розмірність                | Значення                                         |        |                                          |
| ---------- | -------------------------- | ------------------------------------------------ | ------ | ---------------------------------------- |
| $N$        | скаляр                     | Кількість зображень у датасеті                   |        |                                          |
| $x_i$      | $H\times W\times3$         | $i$-те зображення RGB                            |        |                                          |
| $y_i$      | скаляр з діапазону $[1,m]$ | Базова мітка з множини $L$                       |        |                                          |
| $m$        | скаляр                     | Число базових (найдрібніших) класів              |        |                                          |
| $f_{ij}$   | $\mathbb{R}^{d}$           | Вектор ознак $j$-го ROI на $x_i$                 |        |                                          |
| $d$        | скаляр                     | Розмірність простору ознак після $B$             |        |                                          |
| $\mu_l$    | $\mathbb{R}^{d}$           | Центроїд ознак класу $l$                         |        |                                          |
| $\Omega$   | $m\times m$                | Матриця нормованих помилок між парами класів     |        |                                          |
| $M$        | $m\times m$ трикутна       | Матриця комбінованої близькості (3)              |        |                                          |
| $K$        | скаляр                     | Кінцева кількість ієрархічних рівнів             |        |                                          |
| $L_k$      | множина                    | Підмножина класів, що розрізняється на рівні $k$ |        |                                          |
| (          | L\_k                       | )                                                | скаляр | Поточна кількість підкласів на рівні $k$ |
| $k$        | індекс $1\ldots K$         | Номер рівня в ієрархії                           |        |                                          |

## Коротке обґрунтування вигод

| Критерій                    | Ручна схема (2.4–2.5) | Автоматичний алгоритм |
| --------------------------- | --------------------- | --------------------- |
| Вибір рівнів / класів       | емпірично             | формула (4)           |
| Підбір моделей              | фіксований            | дерево рішень (5–6)   |
| Пороги довіри               | не описано            | калібрування 7.1      |
| Підтримка FPS               | декларативно          | перевірка 7.2         |
| Масштабування на нові класи | ручне                 | повторити кроки 1–4   |

Алгоритм робить побудову структури **трасованою, відтворюваною** й адаптивною до нових наборів класів чи апаратних обмежень, зберігаючи концепцію, закладену авторами .



## Крок 5. Призначення моделей кожному рівню $k$ (удосконалене дерево рішень)

Нехай

* $|L_k|$ — кількість підкласів, які треба розрізнити на рівні $k$;
* $\bar{s}_k$ — медіанний розмір ROI (у пікселях) між усіма підкласами рівня;
* $C_{\mathrm{GPU}}$ — доступна обчислювальна потужність (TFLOPS).

| Вхідні умови                               | Детектор $D_k$ | Витяг $F_k$                       | Класифікатор $C_k$                   |                              |                                               |
| ------------------------------------------ | -------------- | --------------------------------- | ------------------------------------ | ---------------------------- | --------------------------------------------- |
| (                                          | L\_k           | !\le!2) та $\bar{s}_k\!>\!48$ px  | **YOLOv11-l** (≈130 FPS на RTX 3090) | Стандартний CSPDarknet-53    | **Linear MLP** (1 прих. шар, 256 нейронів)    |
| (3!\le!                                    | L\_k           | !\le!6) & $\bar{s}_k\!\ge\!32$ px | **YOLOv11-m**                        | CSPDarknet-53 + SPP-FPN      | **FT-Transformer** (d = 512, h = 8, 2 блоки)  |
| (                                          | L\_k           | !>!6) або $\bar{s}_k\!<\!32$ px   | **RT-DETR-Tiny** (ембединг 256)      | CSPDarknet-53 + FPN + CARAFE | **FT-Transformer** (d = 768, h = 12, 4 блоки) |
| При дефіциті TFLOPS ($C_{\mathrm{GPU}}<5$) | **YOLO-Nano**  | Lightweight FPN                   | **MobileNet-MLP**                    |                              |                                               |
"""

def add_run_with_formatting(paragraph, children):
    """Допоміжна функція для обробки вбудованих стилів (жирний, курсив, код)."""
    for child in children:
        if child.type == 'text':
            run = paragraph.add_run(child.content)
        elif child.type == 'strong_open':
            run = paragraph.add_run()
            run.bold = True
        elif child.type == 'strong_close':
            run = paragraph.add_run()
            run.bold = False
        elif child.type == 'em_open':
            run = paragraph.add_run()
            run.italic = True
        elif child.type == 'em_close':
            run = paragraph.add_run()
            run.italic = False
        elif child.type == 'code_inline':
            run = paragraph.add_run(child.content)
            run.font.name = 'Courier New'
        elif child.type in ['math_inline', 'math_block', 'math_block_tag']:
            # Вставляємо формули як звичайний текст.
            # У MS Word їх можна перетворити на професійні формули.
            run = paragraph.add_run(child.content)
            run.font.name = 'Cambria Math' # Спеціальний шрифт для формул
            
def get_plain_text(children):
    """Допоміжна функція для отримання простого тексту з токенів."""
    text = ""
    for child in children:
        if child.type == 'text':
            text += child.content
        elif child.type in ['math_inline', 'math_block']:
             text += child.content
    return text

def markdown_to_docx(md_text, filename):
    """
    Основна функція для конвертації Markdown тексту в DOCX файл.
    """
    # Ініціалізуємо парсер з плагіном для формул
    md = MarkdownIt().use(dollarmath_plugin)
    tokens = md.parse(md_text)
    
    doc = docx.Document()
    # Встановлюємо базові стилі для документу
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Times New Roman'
    font.size = Pt(12)

    table_data = []
    in_table = False
    
    # Ітерація по токенам для створення елементів документу
    i = 0
    while i < len(tokens):
        token = tokens[i]
        
        if token.type == 'heading_open':
            level = int(token.tag[1])
            i += 1
            content = tokens[i].content
            doc.add_heading(content, level=level)
            i += 1 # пропустити heading_close
        
        elif token.type == 'paragraph_open':
            p = doc.add_paragraph()
            i += 1
            # Обробляємо вбудовані елементи (жирний, курсив, код)
            if tokens[i].type == 'inline':
                add_run_with_formatting(p, tokens[i].children)
            i += 1 # пропустити paragraph_close

        elif token.type == 'blockquote_open':
            i += 1
            p_text = ""
            while tokens[i].type != 'blockquote_close':
                if tokens[i].type == 'paragraph_open':
                    i += 1
                    p_text += get_plain_text(tokens[i].children)
                i += 1
            doc.add_paragraph(p_text, style='Quote')

        elif token.type in ['bullet_list_open', 'ordered_list_open']:
            list_style = 'List Bullet' if token.type == 'bullet_list_open' else 'List Number'
            i += 1
            while tokens[i].type not in ['bullet_list_close', 'ordered_list_close']:
                if tokens[i].type == 'list_item_open':
                    i += 1
                    # Шукаємо текст всередині list_item
                    item_text = ""
                    if tokens[i].type == 'paragraph_open':
                        i+=1
                        item_text = get_plain_text(tokens[i].children)
                    p = doc.add_paragraph(item_text, style=list_style)
                    # Відступи для нумерованих списків з двома цифрами (напр. 1.1)
                    if '.' in item_text.split(' ')[0]:
                         p.paragraph_format.left_indent = Inches(0.5)

                i += 1

        elif token.type == 'hr':
            p = doc.add_paragraph()
            p_border = parse_xml(r'<w:pBdr %s><w:bottom w:val="single" w:sz="6" w:space="1" w:color="auto"/></w:pBdr>' % nsdecls('w'))
            p._p.get_or_add_pPr().append(p_border)
        
        elif token.type == 'fence':
            p = doc.add_paragraph(style='Normal')
            p.add_run(token.content).font.name = 'Courier New'
            # Додаємо фон для блоку коду
            shading_elm = parse_xml(r'<w:shd %s w:fill="F1F1F1"/>' % nsdecls('w'))
            p._p.get_or_add_pPr().append(shading_elm)

        elif token.type == 'table_open':
            # Збираємо дані таблиці
            header = []
            rows = []
            i += 1 # thead_open
            i += 1 # tr_open
            while tokens[i].type != 'tr_close':
                if tokens[i].type == 'th_open':
                    i += 1
                    header.append(get_plain_text(tokens[i].children))
                i += 1
            i += 1 # thead_close
            i += 1 # tbody_open
            
            current_row = []
            while tokens[i].type != 'table_close':
                if tokens[i].type == 'tr_open':
                    current_row = []
                elif tokens[i].type == 'tr_close':
                    if current_row:
                        rows.append(current_row)
                elif tokens[i].type == 'td_open':
                    i += 1
                    current_row.append(get_plain_text(tokens[i].children))
                i += 1

            # Створюємо таблицю в DOCX
            if header and rows:
                table = doc.add_table(rows=1, cols=len(header))
                table.style = 'Table Grid'
                hdr_cells = table.rows[0].cells
                for idx, h_text in enumerate(header):
                    cell_p = hdr_cells[idx].paragraphs[0]
                    cell_p.add_run(h_text).bold = True
                
                for r_data in rows:
                    row_cells = table.add_row().cells
                    for idx, c_text in enumerate(r_data):
                        row_cells[idx].text = c_text
            
            # Додаємо пустий абзац після таблиці для відступу
            doc.add_paragraph()

        elif token.type == 'math_block_tag':
            p = doc.add_paragraph()
            add_run_with_formatting(p, [token])
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
        i += 1
    
    # Зберігаємо документ
    doc.save(filename)
    print(f"Файл '{filename}' успішно створено та збережено.")

# --- Запуск конвертації ---
output_filename = "output_document.docx"
markdown_to_docx(markdown_text, output_filename)

Файл 'output_document.docx' успішно створено та збережено.
